###  Created by Luis A. Sanchez-Perez (alejand@umich.edu).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

A logistic regression forward and backward pass. Compares results to analytical computations.

In [1]:
import numpy as np
from graphs.core import Param
from graphs.core import DataHolder
from graphs.core import Graph
from graphs.core import Operation
from graphs.nodes import linear_node
from graphs.nodes import bias_node
from graphs.nodes import sigmoid_node
from graphs.nodes import bce_node
from sklearn import datasets
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

### Loading dataset

In [2]:
dataset = datasets.load_iris()
predictors = dataset['data']
responses = dataset['target'].reshape(-1,1)
responses[responses == 2] = 1
m,d = predictors.shape

### Creating custom computational graph
Here we use our custom toy implemenations and perform one forward and backward pass computing the gradient.

In [3]:
X_node = DataHolder()
y_node = DataHolder()
w_node = Param(shape=(d,1))
b_node = Param(shape=(1,1))

In [4]:
r_node = linear_node(X_node,w_node)
z_node = bias_node(r_node,b_node)
h_node = sigmoid_node(z_node)
J_node = bce_node(h_node,y_node)

In [5]:
g = Graph()
g.build(J_node).initialize().feed({X_node:predictors, y_node:responses})

In [6]:
g.forward().backward()

### Analytical implementations
Here we use the known formulas for logistic regression to compute cost and gradient. Notice that this is possible only because the graph for logistic regression is a really simple one. If we had a more complex graph deriving these formulas analytically will be impossible (or not viable).

In [7]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def compute_cost(w,X,y):
    h = sigmoid(X.dot(w))
    loglikelihood = sum([np.log(prob) if label else np.log(1-prob) for prob,label in zip(h,y)])
    return -loglikelihood

def compute_grad(w,X,y):
    h = sigmoid(X.dot(w))
    grad = (X.T).dot(h - y)
    return grad

In [8]:
X = np.hstack((np.ones((predictors.shape[0],1)), predictors))
w = np.vstack((b_node.value,w_node.value))
y = y_node.value

### Building tensorflow graph
Here we use tensorflow to create the same computational graph and performs a forward and backward pass computing the gradient.

In [9]:
X_tensor = tf.convert_to_tensor(predictors, dtype=tf.float64)
y_tensor = tf.convert_to_tensor(responses, dtype=tf.int64)
w_tensor = tf.Variable(w_node.value, dtype=tf.float64)
b_tensor = tf.Variable(b_node.value, dtype=tf.float64)

In [10]:
def binary_cross_entropy(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    y_true = tf.cast(y_true, dtype=y_pred.dtype)
    return -tf.reduce_sum(
        y_true * tf.math.log(y_pred) + (1. - y_true) * tf.math.log(1 - y_pred)
    )

In [11]:
with tf.GradientTape() as tape:
    r_tensor = tf.matmul(X_tensor, w_tensor)
    z_tensor = tf.add(r_tensor, b_tensor)
    h_tensor = tf.nn.sigmoid(z_tensor)
    J_tensor = binary_cross_entropy(y_tensor, h_tensor)
#     J_tensor = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(
#         logits=z_tensor, labels=tf.cast(y_tensor, z_tensor.dtype)
#     ))
gradients = tape.gradient(J_tensor, [b_tensor, w_tensor])

### Comparisons!

In [12]:
print('Convetional:', compute_cost(w,X,y).ravel())
print('Graph (Custom):', J_node.value.ravel())
print('Tensorflow:', J_tensor.numpy().ravel())

Convetional: [861.34107898]
Graph (Custom): [861.34107898]
Tensorflow: [861.34107898]


In [13]:
print('Conventional:', compute_grad(w,X,y).flatten())

Conventional: [ -99.8646402  -625.51477989 -286.77156947 -490.32975782 -167.53788101]


In [14]:
print('Graph (Custom):', np.hstack((b_node.gradient.ravel(), w_node.gradient.ravel())))

Graph (Custom): [ -99.8646402  -625.51477989 -286.77156947 -490.32975782 -167.53788101]


In [15]:
print('Tensorflow:', np.hstack((gradients[0].numpy().ravel(), gradients[1].numpy().ravel())))

Tensorflow: [ -99.8646402  -625.51477989 -286.77156947 -490.32975782 -167.53788101]
